<a href="https://colab.research.google.com/github/QuangHuy54/question-answering-roberta/blob/main/QuestionAnswering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transformers installation
! pip install transformers datasets transformers[sentencepiece] datasets
!pip install accelerate evaluate
!pip install rouge_score
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.wh

# Question answering

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load SQuAD dataset

In [ ]:
from datasets import load_dataset,Dataset
import pandas as pd

squad = load_dataset("squad", split="train[:]+validation[:]")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


In [ ]:
print(squad)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 98169
})


Split the dataset's `train` split into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [ ]:
from datasets import DatasetDict
squad = squad.train_test_split(test_size=0.2,shuffle=False)
test_validate=squad["test"].train_test_split(test_size=0.5,shuffle=False)
squad = DatasetDict({
    'train': squad['train'],
    'test': test_validate['test'],
    'validate': test_validate['train']})
print(squad)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 78535
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9817
    })
    validate: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9817
    })
})


Save dataset for offline usage:

In [ ]:
for split, dataset in squad.items():
    dataset.to_json(f"../dataset/squad-{split}.json")

Creating json from Arrow format:   0%|          | 0/79 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
squad["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

There are several important fields here:

- `answers`: the starting location of the answer token and the answer text.
- `context`: background information from which the model needs to extract the answer.
- `question`: the question a model should answer.

## Preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=386,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

Apply the preprocessing function over the entire dataset

In [ ]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/78535 [00:00<?, ? examples/s]

Map:   0%|          | 0/9817 [00:00<?, ? examples/s]

Map:   0%|          | 0/9817 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_squad)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 78535
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 9817
    })
    validate: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 9817
    })
})


In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

## Train

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inf

In [ ]:
training_args = TrainingArguments(
    output_dir="roberta-base-squad-1",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    warmup_ratio = 0.2,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validate"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/QuangHuy54/roberta-base-squad-1 into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.943600,0.830470
2,0.699600,0.794548


TrainOutput(global_step=9818, training_loss=1.0699925891831927, metrics={'train_runtime': 12962.2315, 'train_samples_per_second': 12.118, 'train_steps_per_second': 0.757, 'total_flos': 3.094173103531656e+16, 'train_loss': 1.0699925891831927, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/473M [00:00<?, ?B/s]

Upload file runs/Jun29_12-29-45_f804d562d2c4/events.out.tfevents.1688041795.f804d562d2c4.507.0:   0%|         …

To https://huggingface.co/QuangHuy54/roberta-base-squad-1
   a981432..c7c1b20  main -> main

   a981432..c7c1b20  main -> main

To https://huggingface.co/QuangHuy54/roberta-base-squad-1
   c7c1b20..df36b2b  main -> main

   c7c1b20..df36b2b  main -> main



'https://huggingface.co/QuangHuy54/roberta-base-squad-1/commit/c7c1b20a72f1a4144581fcdd7c520730fe8f8c55'

## Save model to local

In [ ]:
from transformers import AutoModelForQuestionAnswering,AutoTokenizer
import torch
model=AutoModelForQuestionAnswering.from_pretrained("QuangHuy54/roberta-base-squad-1")
tokenizer = AutoTokenizer.from_pretrained("QuangHuy54/roberta-base-squad-1")
tokenizer.save_pretrained("../model/")
model.save_pretrained("../model/")

## Evaluate model on test set

In [ ]:
from evaluate import evaluator
from datasets import Dataset,load_dataset
import pandas as pd
task_evaluator = evaluator("question-answering")
data=load_dataset("json",data_files="../dataset/squad-test.json",split="train")
results = task_evaluator.compute(
    model_or_pipeline="../model/",
    data=data,
    metric="squad",
)
print(results)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d582216fb6d2c49f/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


Filter:   0%|          | 0/9817 [00:00<?, ? examples/s]

{'exact_match': 84.99541611490272, 'f1': 91.70086293929828, 'total_time_in_seconds': 116.35430387500014, 'samples_per_second': 84.37161044379106, 'latency_in_seconds': 0.011852327989711739}


## Testing

In [ ]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="../model/")
question_answerer(question=question, context=context)

{'score': 0.9773239493370056, 'start': 93, 'end': 95, 'answer': '13'}